### 载入所需的库
我们依旧会用gensim去做word2vec的处理，会用sklearn当中的SVM进行建模

In [1]:
# -*- coding: utf-8 -*-
"""
Created on 2015.12.09

@author: Hanxiaoyang
"""
from sklearn.cross_validation import train_test_split
from gensim.models.word2vec import Word2Vec
import numpy as np
import pandas as pd
import jieba
from sklearn.externals import joblib#模型持久化模块，将模型保存至硬盘
from sklearn.svm import SVC
import sys  
#reload(sys)  
#sys.setdefaultencoding('utf8')
#不需要sys.setdefaultencoding("utf-8")这段代码

d:\program files (x86)\python\python35\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
d:\program files (x86)\python\python35\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### 载入数据，做预处理(分词)，切分训练集与测试集

In [4]:
#因为Excel需要单独的模块支持，所以需要安装xlrd模块
#install xlrd
neg=pd.read_excel('data/neg.xls',header=None,index=None)
pos=pd.read_excel('data/pos.xls',header=None,index=None)#每行是一条评论文本
print("neg data:")
print(neg.head(5))
print("pos data:")
print(pos.head(5))

print("type of neg data=",type(neg))
print("type of pos data=",type(pos))

#下面进行批量分词，较为耗时
cw = lambda x: list(jieba.cut(x))#内部分词是用HMM
#pos[0]返回columns为0的列，注意这种方式一次只能返回一个列。pos.0与pos['0']意思一样
pos['words'] = pos[0].apply(cw)#对第一列，因为只有一列
neg['words'] = neg[0].apply(cw)#在原来的对象基础上再增加一个words这一个新列
print(pos['words'])#查看正样本的分词结果

neg data:
                                                   0
0  做为一本声名在外的流行书，说的还是广州的外企，按道理应该和我的生存环境差不多啊。但是一看之下...
1  作者有明显的自恋倾向，只有有老公养不上班的太太们才能像她那样生活。很多方法都不实用，还有抄袭...
2  作者完全是以一个过来的自认为是成功者的角度去写这个问题，感觉很不客观。虽然不是很喜欢，但是，...
3       作者提倡内调，不信任化妆品，这点赞同。但是所列举的方法太麻烦，配料也不好找。不是太实用。
4                  作者的文笔一般，观点也是和市面上的同类书大同小异，不推荐读者购买。
pos data:
                                                   0
0  做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持一...
1  作者真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到...
2  作者长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产...
3  作者在战几时之前用了＂拥抱＂令人叫绝．日本如果没有战败，就有会有美军的占领，没胡官僚主义的延...
4  作者在少年时即喜阅读，能看出他精读了无数经典，因而他有一个庞大的内心世界。他的作品最难能可贵...
type of neg data= <class 'pandas.core.frame.DataFrame'>
type of pos data= <class 'pandas.core.frame.DataFrame'>
0        [做, 父母, 一定, 要, 有, 刘墉, 这样, 的, 心态, ，, 不断, 地, 学习,...
1        [作者, 真有, 英国人, 严谨, 的, 风格, ，, 提出, 观点, 、, 进行, 论述,...
2        [作者, 长篇大论, 借用, 详细, 报告, 数据处理, 工作, 和, 计算结果, 支持, ...
3        [作者, 在, 战, 几时, 之前, 用, 了, ＂, 拥抱, ＂, 令人, 叫绝, ．, 

In [6]:
print(type(pos))
print(pos.head(2))#返回前2行的数据

<class 'pandas.core.frame.DataFrame'>
                                                   0  \
0  做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持一...   
1  作者真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到...   

                                               words  
0  [做, 父母, 一定, 要, 有, 刘墉, 这样, 的, 心态, ，, 不断, 地, 学习,...  
1  [作者, 真有, 英国人, 严谨, 的, 风格, ，, 提出, 观点, 、, 进行, 论述,...  


In [9]:
print(pos[0:3])#返回前3行的数据，列名分别是0，words

                                                   0  \
0  做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持一...   
1  作者真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到...   
2  作者长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产...   

                                               words  
0  [做, 父母, 一定, 要, 有, 刘墉, 这样, 的, 心态, ，, 不断, 地, 学习,...  
1  [作者, 真有, 英国人, 严谨, 的, 风格, ，, 提出, 观点, 、, 进行, 论述,...  
2  [作者, 长篇大论, 借用, 详细, 报告, 数据处理, 工作, 和, 计算结果, 支持, ...  


In [11]:
print(np.ones(len(pos)).shape)

(10677,)


In [12]:
#将上述的操作集成一个函数
def load_file_and_preprocessing():
    neg=pd.read_excel('data/neg.xls',header=None,index=None)
    pos=pd.read_excel('data/pos.xls',header=None,index=None)
    
    cw = lambda x: list(jieba.cut(x))
    pos['words'] = pos[0].apply(cw)
    neg['words'] = neg[0].apply(cw)

    #print pos['words']
    #use 1 for positive sentiment, 0 for negative
    y = np.concatenate((np.ones(len(pos)), np.zeros(len(neg))))#默认是沿着y轴,即轴向拼接
    #此时的y矩阵记录的情感的结果值，1表示正面，0表示负面
    print("y size=",y.shape)
    #Sklearn-train_test_split随机划分训练集和测试集
    x_train, x_test, y_train, y_test = train_test_split(np.concatenate((pos['words'], neg['words'])), y, test_size=0.2,  random_state=1)
    #划分测试集和训练集。在这里严格意义说是验证集，并不是测试集。
    
    #random_state：是随机数的种子。
    #随机数种子：其实就是该组随机数的编号，在需要重复试验的时候，保证得到一组一样的随机数。
    #比如你每次都填1，其他参数一样的情况下你得到的随机数组是一样的。但填0或不填，每次都会不一样。
    #随机数的产生取决于种子，随机数和种子之间的关系遵从以下两个规则：
    #种子不同，产生不同的随机数；种子相同，即使实例不同也产生相同的随机数。
    
    np.save('svm_data/y_train.npy',y_train)#先存储下训练集
    np.save('svm_data/y_test.npy',y_test)
    return x_train,x_test

### 对每个句子的所有词向量取均值，来生成一个句子的vector

In [13]:
def build_sentence_vector(text, size,imdb_w2v):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in text:
        try:
            vec += imdb_w2v[word].reshape((1, size))#向量逐点求和
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

### 计算词向量

In [24]:
#计算词向量
def get_train_vecs(x_train,x_test):
    n_dim = 300
    #初始化模型和词表
    imdb_w2v = Word2Vec(size=n_dim, min_count=10)
    #其实参数很多，min_count表示对字典做截断。词频少于min_count次数的单词会被丢弃掉, 默认值为5
    imdb_w2v.build_vocab(x_train)#收集单词及其词频来构建一个内部字典树结构
    
    #在评论训练集上建模(可能会花费几分钟)total_examples=model.corpus_count
    imdb_w2v.train(x_train,total_examples=imdb_w2v.corpus_count, epochs=imdb_w2v.iter)#训练神经网络
    #train函数的调用可能由于版本
    
    train_vecs = np.concatenate([build_sentence_vector(z, n_dim,imdb_w2v) for z in x_train])
    #train_vecs = scale(train_vecs)
    
    np.save('svm_data/train_vecs.npy',train_vecs)
    print(train_vecs.shape)
    #在测试集上训练
    imdb_w2v.train(x_test, total_examples=imdb_w2v.corpus_count, epochs=imdb_w2v.iter)
    #对于测试集的训练，如果只给定测试数据集，而不指定参数，会报错如下：
    #ValueError: You must specify either total_examples or total_words, for proper alpha and progress calculations. 
    #The usual value is total_examples=model.corpus_count.
    imdb_w2v.save('svm_data/w2v_model/w2v_model.pkl')
    #Build test tweet vectors then scale
    test_vecs = np.concatenate([build_sentence_vector(z, n_dim,imdb_w2v) for z in x_test])
    #test_vecs = scale(test_vecs)
    np.save('svm_data/test_vecs.npy',test_vecs)
    print(test_vecs.shape)

In [21]:
x_train,x_test = load_file_and_preprocessing()

#train_vecs.npy
#w2v_model.pkl
#test_vecs.npy都进行保存 


y size= (21105,)


In [25]:
get_train_vecs(x_train,x_test)

d:\program files (x86)\python\python35\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


(16884, 300)


(4221, 300)


In [26]:
def get_data():
    train_vecs=np.load('svm_data/train_vecs.npy')
    y_train=np.load('svm_data/y_train.npy')
    test_vecs=np.load('svm_data/test_vecs.npy')
    y_test=np.load('svm_data/y_test.npy') 
    return train_vecs,y_train,test_vecs,y_test

In [27]:
train_vecs,y_train,test_vecs,y_test = get_data()

### 训练svm模型

In [29]:
def svm_train(train_vecs,y_train,test_vecs,y_test):
    clf=SVC(kernel='rbf',verbose=True)
    clf.fit(train_vecs,y_train)
    joblib.dump(clf, 'svm_data/svm_model/model.pkl')
    print(clf.score(test_vecs,y_test))

In [30]:
tic = time.time()
svm_train(train_vecs,y_train,test_vecs,y_test)#稍微耗时,5分钟左右
toc = time.time()
print("cost time=",toc-tic)

[LibSVM]0.809049988154


### 构建待预测句子的向量

In [32]:
def get_predict_vecs(words):
    n_dim = 300
    imdb_w2v = Word2Vec.load('svm_data/w2v_model/w2v_model.pkl')
    #imdb_w2v.train(words)
    train_vecs = build_sentence_vector(words, n_dim,imdb_w2v)
    #print train_vecs.shape
    return train_vecs

### 对单个句子进行情感判断 

In [33]:
def svm_predict(string):
    words=jieba.lcut(string)
    words_vecs=get_predict_vecs(words)#对输入句子转为向量
    clf=joblib.load('svm_data/svm_model/model.pkl')#加载训练好的情感分析模型
     
    result=clf.predict(words_vecs)#对输入句子进行情感预测
    
    if int(result[0])==1:
        print(string,' positive')
    else:
        print(string,' negative')

In [50]:
##对输入句子情感进行判断
#string='电池充完了电连手机都打不开.简直烂的要命.真是金玉其外,败絮其中!连5号电池都不如'
#string='牛逼的手机，从3米高的地方摔下去都没坏，质量非常好'    
#string='从3米高的地方摔下去都没坏，还能够替我挡子弹啊'    正确
#string='从3米高的地方摔下去就坏了'
string='能够挡子弹'
svm_predict(string)

从3米高的地方摔下去就坏了  negative


d:\program files (x86)\python\python35\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
